In [1]:
import pandas as pd

# Python standard library imports
import time

# Third-party imports for database connection and data manipulation
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report

In [2]:
# Database connection parameters
dbname = 'train'
user = 'narmina'
password = 'narmina7877'
host = 'localhost'  # localhost or the server address
port = '5433'  # default PostgreSQL port is 5432

# Establish a connection to the database
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

In [3]:
# Define the query with placeholders for parameters
query1 = f"""
SELECT rs_e_rpm_pc1, rs_e_inairtemp_pc1,rs_e_wattemp_pc1, rs_t_oiltemp_pc1 
FROM vehicle_data where rs_e_rpm_pc1<>0 and 
((rs_e_inairtemp_pc1 > 65 or rs_e_inairtemp_pc1 <0) or 
(rs_e_wattemp_pc1 > 100 or rs_e_wattemp_pc1 < 0) or 
(rs_t_oiltemp_pc1 > 115 or rs_t_oiltemp_pc1 < 0));
"""

# Start timing
start_time = time.time()

# Execute the query and fetch the data into a DataFrame
df1 = pd.read_sql_query(query1, engine)

# End timing
end_time = time.time()
print(f"Query took {end_time - start_time} seconds to run.")
# Close the database connection
engine.dispose()

Query took 5.952168941497803 seconds to run.


In [4]:
print(df1)

      rs_e_rpm_pc1  rs_e_inairtemp_pc1  rs_e_wattemp_pc1  rs_t_oiltemp_pc1
0           1888.0                47.0             101.0              98.0
1            189.0                43.0             103.0             101.0
2           1497.0                47.0             102.0             103.0
3            806.0                43.0             101.0             101.0
4           1497.0                45.0             102.0             101.0
...            ...                 ...               ...               ...
3207         801.0                45.0             102.0             102.0
3208        1399.0                66.0              82.0              84.0
3209        1461.5                65.5              81.0              84.0
3210        1809.0                66.0              85.0              82.0
3211        1879.0                67.0              82.0              81.0

[3212 rows x 4 columns]


In [5]:
# Define the query with placeholders for parameters
query2 = f"""
SELECT rs_e_rpm_pc2, rs_e_inairtemp_pc2,rs_e_wattemp_pc2, rs_t_oiltemp_pc2 
FROM vehicle_data where rs_e_rpm_pc2<>0 and 
((rs_e_inairtemp_pc2 > 65 or rs_e_inairtemp_pc2 <0) 
or (rs_e_wattemp_pc2 > 100 or rs_e_wattemp_pc2 < 0) 
or (rs_t_oiltemp_pc2 > 115 or rs_t_oiltemp_pc2 < 0));
"""

# Start timing
start_time = time.time()

# Execute the query and fetch the data into a DataFrame
df2 = pd.read_sql_query(query2, engine)

# End timing
end_time = time.time()
print(f"Query took {end_time - start_time} seconds to run.")
# Close the database connection
engine.dispose()

Query took 5.637232780456543 seconds to run.


In [6]:
print(df2)

      rs_e_rpm_pc2  rs_e_inairtemp_pc2  rs_e_wattemp_pc2  rs_t_oiltemp_pc2
0           1574.5                66.0              81.5              85.0
1           1845.0                67.0              93.0              92.0
2           1496.0                47.0             101.0              99.0
3           1844.0                66.0              88.0              92.0
4           1503.0                66.0              88.0              89.0
...            ...                 ...               ...               ...
2995        1807.0                43.0             102.0              98.0
2996         802.0                45.0             103.0              98.0
2997        1807.0                68.0              87.0              92.0
2998          34.0                73.0              66.0              71.5
2999         799.0                46.0             105.0             100.0

[3000 rows x 4 columns]
